In [1]:
!pip install plotly
!pip install yfinance

In [3]:
# librerías necesarias
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Dropout, Bidirectional, Conv1D, MaxPooling1D, Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

# Descargamos datos históricos de ECOPETROL desde 2009 hasta abril 2025
df = yf.download('EC', start='2009-01-01', end='2025-04-22')
df = df[['Close']].dropna()  # Solo nos interesa la columna "Close" y eliminamos valores nulos

# Escalamos los precios entre 0 y 1 para que la red neuronal aprenda mejor
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['Close']])

# Función para crear ventanas de datos secuenciales (X: entrada, y: salida esperada)
def crear_secuencias(data, ventana):
    X, y = [], []
    for i in range(ventana, len(data)):
        X.append(data[i-ventana:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Definimos el tamaño de ventana (60 días anteriores para predecir el siguiente)
ventana = 60
X, y = crear_secuencias(scaled_data, ventana)
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape necesario para RNNs (samples, timesteps, features)

# Dividimos en conjuntos de entrenamiento (80%) y prueba (20%)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Definimos un stop para evitar sobreajuste (por si no mejora la pérdida)
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Función genérica para entrenar, predecir, evaluar y graficar resultados
def entrenar_y_predecir(modelo, nombre_modelo, optimizador):
    modelo.compile(optimizer=optimizador, loss='mean_squared_error')  # Compilamos el modelo
    modelo.fit(
        X_train, y_train,
        epochs=50, batch_size=32,
        verbose=0,
        callbacks=[early_stop],
        validation_split=0.1
    )
    # Hacemos predicciones sobre el conjunto de prueba
    pred = modelo.predict(X_test)
    pred_inv = scaler.inverse_transform(pred)  # Invertimos la escala para interpretar en precios reales
    y_test_inv = scaler.inverse_transform(y_test)

    # Calculamos métricas de error
    rmse = np.sqrt(mean_squared_error(y_test_inv, pred_inv))
    mae  = mean_absolute_error(y_test_inv, pred_inv)
    r2   = r2_score(y_test_inv, pred_inv)

    # Imprimimos resultados
    print(f'\n {nombre_modelo}')
    print(f'   RMSE: {rmse:.4f}')
    print(f'   MAE:  {mae:.4f}')
    print(f'   R²:   {r2:.4f}')

    # Predicción de un día futuro
    entrada = scaled_data[-ventana:].reshape(1, ventana, 1)
    futura = modelo.predict(entrada)
    futura = scaler.inverse_transform(futura)[0][0]
    fecha_futura = df.index[-1] + pd.Timedelta(days=1)

    # Visualizamos resultados en gráfica interactiva
    trace_real = go.Scatter(
        x=df.index[-len(y_test):],
        y=y_test_inv.flatten(),
        mode='lines', name='Valor Real'
    )
    trace_pred = go.Scatter(
        x=df.index[-len(y_test):],
        y=pred_inv.flatten(),
        mode='lines', name='Predicción'
    )
    trace_line = go.Scatter(
        x=[df.index[-1], df.index[-1]],
        y=[y_test_inv.min(), y_test_inv.max()],
        mode='lines', name='Inicio Futuro',
        line=dict(color='red', dash='dash')
    )
    trace_fut = go.Scatter(
        x=[fecha_futura],
        y=[futura],
        mode='markers+text', name='Predicción Futura',
        marker=dict(color='green', size=10),
        text=[f"{futura:.2f}"], textposition="top center"
    )

    fig = go.Figure(data=[trace_real, trace_pred, trace_line, trace_fut])
    fig.update_layout(
        title=f'{nombre_modelo} - Predicción Precio ECOPETROL',
        xaxis_title='Fecha', yaxis_title='Precio',
        template='plotly_white',
        legend=dict(x=0.01, y=0.99)
    )
    fig.show()

# MODELOS ORIGINALES

# Modelo LSTM básico
modelo_lstm_basico = Sequential([
    Input(shape=(ventana, 1)),
    LSTM(50, return_sequences=True),
    LSTM(50),
    Dense(1)
])
entrenar_y_predecir(modelo_lstm_basico, 'LSTM Básico', Adam(learning_rate=0.001))

# Modelo RNN simple básico
modelo_rnn_basico = Sequential([
    Input(shape=(ventana, 1)),
    SimpleRNN(50, return_sequences=True),
    SimpleRNN(50),
    Dense(1)
])
entrenar_y_predecir(modelo_rnn_basico, 'RNN Simple Básico', Adam(learning_rate=0.001))

# Modelo BPTT básico
modelo_bptt_basico = Sequential([
    Input(shape=(ventana, 1)),
    SimpleRNN(50, return_sequences=True),
    SimpleRNN(50, return_sequences=True),
    SimpleRNN(50),
    Dense(1)
])
entrenar_y_predecir(modelo_bptt_basico, 'BPTT Simulado Básico', Adam(learning_rate=0.001))

# MODELOS MEJORADOS

# Modelo LSTM mejorado
modelo_lstm_mejorado = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    LSTM(80, return_sequences=True),
    Dropout(0.1),
    LSTM(60),
    Dense(1, kernel_regularizer=regularizers.l2(0.001))
])
entrenar_y_predecir(modelo_lstm_mejorado, 'LSTM Mejorado', Adam(learning_rate=0.001))

# Modelo RNN simple mejorado
modelo_rnn_mejorado = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    SimpleRNN(80, return_sequences=True),
    Dropout(0.1),
    SimpleRNN(60),
    Dense(1, kernel_regularizer=regularizers.l2(0.001))
])
entrenar_y_predecir(modelo_rnn_mejorado, 'RNN Simple Mejorado', Adam(learning_rate=0.001))

# Modelo BPTT mejorado con más capas, Dropout y regularización
modelo_bptt_mejorado = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    SimpleRNN(60, return_sequences=True),
    Dropout(0.1),
    SimpleRNN(60, return_sequences=True),
    Dropout(0.1),
    SimpleRNN(60),
    Dense(1, kernel_regularizer=regularizers.l2(0.001))
])
entrenar_y_predecir(modelo_bptt_mejorado, 'BPTT Simulado Mejorado', Adam(learning_rate=0.001))


[*********************100%***********************]  1 of 1 completed


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step

 LSTM Básico
   RMSE: 0.2379
   MAE:  0.1797
   R²:   0.9739
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step

 RNN Simple Básico
   RMSE: 0.2651
   MAE:  0.2032
   R²:   0.9675
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step

 BPTT Simulado Básico
   RMSE: 0.5035
   MAE:  0.4515
   R²:   0.8829
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step

 LSTM Mejorado
   RMSE: 0.2947
   MAE:  0.2186
   R²:   0.9599
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step

 RNN Simple Mejorado
   RMSE: 0.3500
   MAE:  0.2877
   R²:   0.9434
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step

 BPTT Simulado Mejorado
   RMSE: 0.3272
   MAE:  0.2552
   R²:   0.9505
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


In [4]:
# NUEVAS ARQUITECTURAS

# Bidirectional LSTM
modelo_lstm_bi = Sequential([
    Input(shape=(ventana, 1)),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(64)),
    Dropout(0.2),
    Dense(1)
])
entrenar_y_predecir(modelo_lstm_bi, 'Bidirectional LSTM', RMSprop(learning_rate=0.0001))

# Conv1D + LSTM
modelo_conv1d_lstm = Sequential([
    Input(shape=(ventana, 1)),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(64),
    Dropout(0.2),
    Dense(1)
])
entrenar_y_predecir(modelo_conv1d_lstm, 'Conv1D + LSTM', RMSprop(learning_rate=0.0001))

26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step

 Bidirectional LSTM
   RMSE: 0.4373
   MAE:  0.3253
   R²:   0.9116
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step

 Conv1D + LSTM
   RMSE: 0.4351
   MAE:  0.3122
   R²:   0.9125
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


# Análisis de modelos predictivos para el precio de ECOPETROL

## Justificación de arquitecturas elegidas

Se propusieron diferentes modelos de redes neuronales recurrentes (RNN) y LSTM:

- **Modelo LSTM básico**: dos capas LSTM de 50 unidades cada una. Se eligió por su capacidad de capturar dependencias a largo plazo, cruciales en series financieras.
- **Modelo RNN simple básico**: dos capas SimpleRNN, usado como línea base de comparación, más simple pero efectivo en series menos complejas.
- **Modelo BPTT simulado básico**: tres capas SimpleRNN, buscando capturar patrones más profundos a través de la técnica de Backpropagation through time.

**Mejoras aplicadas:**
- Uso de Dropout para mitigar sobreajuste en las versiones mejoradas.
- Incremento de unidades por capa (por ejemplo, 60 unidades en LSTM mejorado).
- Cambio de optimizador de Adam (0.001) a RMSprop (0.0005) en versiones mejoradas, para observar su impacto.

**Nuevas arquitecturas propuestas:**
- **Bidirectional LSTM**: para capturar patrones tanto hacia adelante como hacia atrás en la serie.
- **Conv1D + LSTM**: integración de extracción local de características (Conv1D) con procesamiento secuencial (LSTM).

Se quiso comprobar si una arquitectura híbrida o más profunda mejora realmente el desempeño o si basta con arquitecturas más simples bien configuradas.


## Análisis de resultados

### Comparación de métricas

| Modelo | RMSE | MAE | R² |
|:---|:---|:---|:---|
| **LSTM básico** | 0.2379 | 0.1797 | 0.9739 |
| **RNN simple básico** | 0.2651 | 0.2032 | 0.9675 |
| **BPTT simulado básico** | 0.5035 | 0.4515 | 0.8829 |
| **LSTM mejorado** | 0.2947 | 0.2186 | 0.9599 |
| **RNN simple mejorado** | 0.3500 | 0.2877 | 0.9434 |
| **BPTT simulado mejorado** | 0.3272 | 0.2552 | 0.9505 |
| **Bidirectional LSTM** | 0.4373 | 0.3253 | 0.9116 |
| **Conv1D + LSTM** | 0.4351 | 0.3122 | 0.9125 |

### Principales observaciones:

- El LSTM básico logra las mejores métricas generales, con el menor RMSE y MAE y un R² de 0.9739.
- **RNN simple básico** también tiene buen desempeño, aunque ligeramente inferior al LSTM Básico.
- Los modelos mejorados, pese a usar Dropout y cambios de optimizador, **no superan** a las versiones básicas.
- El BPTT simulado básico muestra el peor desempeño entre los modelos básicos.
- Las nuevas arquitecturas (**Bidirectional LSTM** y **Conv1D + LSTM**) no mejoran los resultados frente al modelo LSTM básico.

Aunque en otros contextos añadir regularización y capas adicionales ayuda, aquí parece que los modelos simples se adaptan mejor a la naturaleza del problema y al tipo de datos utilizado.

### Análisis visual de predicciones

- Los modelos básicos (LSTM y RNN) siguen mejor el comportamiento de la serie, capturando picos, valles y fluctuaciones.
- Los modelos mejorados y nuevos tienden a "suavizar" en exceso, perdiendo detalle en las zonas más volátiles.

En datos de tipo financiero, creo que la volatilidad es importante, por lo que suavizar demasiado puede hacer que el modelo no reaccione correctamente a cambios abruptos en el mercado.

## Comparación Técnica y Conceptual

### Rendimiento por familia de modelos

- Modelos básicos superan a los mejorados en todas las métricas.
- Se observa subajuste en los modelos mejorados, posiblemente debido al uso excesivo de Dropout.
- Riesgo de sobreajuste en modelos básicos, dado que no se aplicó validación cruzada exhaustiva.

### RNN vs LSTM

- LSTM Básico supera a RNN Simple Básico, aunque por una diferencia moderada.
- Esto refuerza la ventaja de LSTM en series donde las dependencias temporales a largo plazo son relevantes.

### Arquitecturas nuevas

- **Bidirectional LSTM**: RMSE de 0.4373, MAE de 0.3253, y R² de 0.9116.
- **Conv1D + LSTM**: RMSE de 0.4351, MAE de 0.3122, y R² de 0.9125.
- Aunque ambas nuevas arquitecturas capturan patrones temporales complejos, no mejoran el desempeño respecto a modelos más sencillos.

A pesar de integrar técnicas avanzadas como bidireccionalidad o extracción local de características, los resultados indican que la dinámica temporal del precio de ECOPETROL puede ser modelada eficazmente mediante arquitecturas más simples.

### Análisis de optimizadores

- El uso de RMSprop en modelos mejorados **no** mejoró los resultados.
- Adam mostró ser más efectivo en este tipo de problema.


## Reflexión crítica

### Limitaciones

- No se usó validación cruzada, por lo que el sobreajuste no puede ser descartado totalmente.
- Dropout pudo ser excesivo (valor 0.2); podría explorarse un Dropout menor (0.1) en próximas corridas.
- Ventana fija de 60 días: puede no ser óptima para todos los modelos.
- Factores externos (como precios internacionales del petróleo) no fueron incluidos en el modelo.

### Posibilidades de mejora

- Ajustar la tasa de Dropout y otros hiperparámetros.
- Incorporar indicadores técnicos como RSI, MACD, etc para comprender mejor este tipo de datos financieros.
- Probar arquitecturas basadas en Transformers para series temporales.
- Utilizar backtesting con ventanas deslizantes.
- Considerar métricas financieras como el retorno direccional o el Sharpe ratio.

## Conclusiones

- El modelo LSTM Básico resultó ser el más efectivo en este análisis.
- La simplicidad, combinada con una arquitectura adecuada y el optimizador correcto, superó a versiones más sofisticadas.
- La regularización excesiva y cambios de optimizador afectaron negativamente el desempeño de los modelos mejorados.
- Se destaca la importancia de validar rigurosamente cada modificación arquitectónica y adaptarla a la naturaleza específica de los datos.
---
- Demasiado Dropout puede hacer que el modelo aprenda muy poco.
- Adam es generalmente más rápido y efectivo en datasets medianos; RMSprop es útil en otros contextos, pero aquí no ayudó.
- No toda arquitectura sofisticada mejora el resultado. Depende del tipo de serie y de su comportamiento.

